In [ ]:
# install and load packages, you only need to do this once. 

rprofile_path = file.path(Sys.getenv("HOME"), ".Rprofile")
write('\noptions(repos=c(getOption(\'repos\'),
          CRAN=\'https://cloud.r-project.org\',
          USGS=\'https://owi.usgs.gov/R\'))\n',
      rprofile_path, 
      append =  TRUE)
install.packages('smwrBase')
install.packages('rloadest')
install.packages('car')
install.packages('unitted')
install.packages('dplyr')
packageurl <- "https://github.com/USGS-R/loadflex/archive/v1.0.1.tar.gz"
install.packages(packageurl, repos=NULL, type="source")
install.packages('dplyr')


library('loadflex')
library('rloadest')

In [ ]:


newnames <- as.vector(t(read.table(
  "MPRDaily.csv",
  sep=",",header=FALSE,nrows=1,skip=25,stringsAsFactors=FALSE)[1,]))

MPRDailyDischarge <- read.table(
  "MPRDaily.csv",
  sep=",",header=FALSE,col.names=newnames,skip=28,stringsAsFactors=FALSE)
#MPR_SSForLoadest-1991.csv

MPRDailyDischarge$DateTime <- as.POSIXct(strptime(MPRDailyDischarge$datetime,format="%m/%d/%Y" ,tz="America/Puerto_Rico"))
MPRDailyDischarge$date<-  as.Date(MPRDailyDischarge$date, format("%m/%d/%Y"),tz="America/Puerto_Rico")
#MPRDischargeShort <- MPRDischarge[complete.cases(MPRDischarge["CFS"]),]




newnames <- as.vector(t(read.table(
  "MPRDischargefrom2009.csv",
  sep=",",header=FALSE,nrows=1,stringsAsFactors=FALSE)[1,]))

MPRDischarge <- read.table(
  "MPRDischargefrom2009.csv", #MPR1991-1996
  sep=",",header=FALSE,col.names=newnames,skip=1,stringsAsFactors=FALSE)
#MPR_SSForLoadest-1991.csv
MPRDischarge$DateTime <- as.POSIXct(strptime(MPRDischarge$DateTime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))  #as.Date(MPRDischarge$datetime, format="%m/%d/%Y %H:%M")
#MPRDischarge$date<- as.POSIXct(strptime(MPRDischarge$datetime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))
MPRDischarge$date <- as.Date(MPRDischarge$Date, format("%m/%d/%Y"),tz="America/Puerto_Rico")
MPRDischarge$CFS <- MPRDischarge$USGS_cfps
#MPRDischarge <- MPRDischarge[complete.cases(MPRDischarge["DateTime"]),]


daterange <- range(MPRDischarge$DateTime)

MPRDischargeInterp <- data.frame(date=seq(daterange[1],daterange[2],by=15*60))
MPRDischargeInterp$DateTime <- MPRDischargeInterp$date
MPRDischargeInterp$date <- as.Date(MPRDischargeInterp$date, format("%m/%d/%Y"),tz="America/Puerto_Rico")
# MPRDischargeShortInterp$CFS_Approx <- 
#   approx(
#     x=MPRDailyDischarge$date,
#     y=MPRDailyDischarge$CFS,
#     xout=MPRDailyDischarge$date,
#     rule=2
#   )$y
MPRTimeSeqWithDaily <- merge(MPRDischargeInterp,MPRDailyDischarge, by=c('date'),all.x=TRUE)
MPRTimeSeqWithDaily$DateTime <- MPRTimeSeqWithDaily$DateTime.x
MPRDischargeWithDaily<- merge(MPRTimeSeqWithDaily,MPRDischarge, by=c('DateTime'),all.x=TRUE)

#inspect the time period with a gap in the 15 minute record
MPRDischargeWithDailyShort <-MPRDischargeWithDaily[MPRDischargeWithDaily$DateTime.y >= as.POSIXct("2009-03-24")  
                                                                 & MPRDischargeWithDaily$DateTime.y <= as.POSIXct("2009-03-26"),]

#if 15 min record is NA fill it with daily record 

MPRDischargeWithDaily2 <- within(MPRDischargeWithDaily, CFS.y<- ifelse(is.na(CFS.y), CFS.x, CFS.y))
#inspect the time period with a gap in the 15 minute record
MPRDischargeWithDailyShort2 <-MPRDischargeWithDaily2[MPRDischargeWithDaily2$DateTime.y >= as.POSIXct("2009-03-24") 
                                                                   & MPRDischargeWithDaily2$DateTime.y <= as.POSIXct("2009-03-26"),]

MPRDischargeShort <- data.frame(MPRDischargeWithDaily2$DateTime,MPRDischargeWithDaily2$CFS.y)
MPRDischargeShort<- rename(MPRDischargeShort, c("MPRDischargeWithDaily2.DateTime"="DateTime", "MPRDischargeWithDaily2.CFS.y"="CFS"))
#MPRDischargeShort<-unique(MPRDischargeShort$DateTime)
MPRDischargeShort<-MPRDischargeShort[!duplicated(MPRDischargeShort), ]
#MPRDischargeShortnotcomplete <- MPRDischargeShort[!complete.cases(MPRDischargeShort["CFS"]),]


newnames <- as.vector(t(read.table(
  "McD MPR Query.csv",
  sep=",",header=FALSE,nrows=1,stringsAsFactors=FALSE)[1,]))

MPR_chem <- read.table(
  "McD MPR Query.csv", 
  sep=",",header=FALSE,col.names=newnames,skip=1,stringsAsFactors=FALSE)


MPR_chem$date <- as.POSIXct(strptime(MPR_chem$Sample_Date  ,format="%m/%d/%Y",tz="America/Puerto_Rico"))
#MPR_chem$date <- as.Date(MPR_chem$date, format="%d/%m/%Y")
#MPRDischarge$DateTime <- as.POSIXct(strptime(MPRDischarge$DateTime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))  #as.Date(MPRDischarge$datetime, format="%m/%d/%Y %H:%M")
MPRDischargeShort$date<- MPRDischargeShort$DateTime #as.POSIXct(strptime(MPRDischarge$DateTime,format="%m/%d/%Y %H:%M" ,tz="America/Puerto_Rico"))

#MPR_chemNOCFS <-MPR_chem[is.na(MPR_chem$CFS),] 
#matchingdates <- which(abs(MPR_chemNOCFS$date-MPRDischargeShort$date) == min(abs(MPR_chemNOCFS$date-MPRDischargeShort$date)))                                                     

#get CFS for field data from the same source as our observed discharge
MPR_chem$CFS <- approx(
  x=as.numeric(MPRDischargeShort$date),
  y=MPRDischargeShort$CFS,
  xout=as.numeric(MPR_chem$date)#,rule=2
)$y


#ggplot() +geom_line(data= MPRDischargeShort, aes(x=date, y=log(CFS),  color="cfs")) +geom_point(data= MPR_chemNa, aes(x=date, y=log(CFS),  color="cfs Observed")) 


#MPRDischargeShort <-  MPRDischarge[MPRDischarge$date == as.POSIXct("2011-04-12"),]
#Na
MPR_chemNa <- MPR_chem[complete.cases(MPR_chem["Na_mg_L"]),]
MPR_chemNa <- MPR_chemNa[complete.cases(MPR_chemNa["CFS"]),]

#MPRDischarge <- MPRDischarge[c("DateTime","CFS")]

meta <- metadata(constituent="Na_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1")


lr <- loadReg2(loadReg(Na_mg_L ~ model(1), data=MPR_chemNa,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

#create the rloadest report for Na
lrforreport <- loadReg(Na_mg_L ~ model(1), data=MPR_chemNa,
        flow="CFS", dates="date", time.step="instantaneous", 
        flow.units="cfs", conc.units="mg/L", load.units="kg")
loadReport(lrforreport,"NaMPR")

lc <- loadComp(reg.model=lr, interp.format="flux", 
                   interp.data=MPR_chemNa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

#wis_reg <- loadReg2(loadReg(NO3_corrected_mgL ~ model(9), data=wisregdat,
#                            flow="Q_cms", dates="Date", time.step="instantaneous", 
#                            flow.units="cms", conc.units="mg/L", load.units="kg", station="Lamprey at Wiswall Dam, NH"))
#wis_cmp <- loadComp(reg.model=wis_reg, interp.format="conc", interp.data=wisintdat)


#wis_sensordata$Date <- as.POSIXct(strptime(wis_sensordata$Date, "%Y-%m-%d %H:%M:%S"))


#MPRDischarge2012short <- MPRDischargeShort[MPRDischargeShort$DateTime >= as.POSIXct("2009-1-01") 
#                                               & MPRDischargeShort$DateTime <= as.POSIXct("2011-12-31"),]
#preds_lrNaConc <- predictSolute(lr, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)
#preds_lcNaConc <- predictSolute(lc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)


#preds_lrNa <- predictSolute(lr, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

#aggs_lrNa <- aggregateSolute(preds_lrNa,meta, format="flux total", se.preds=preds_lrNa$se.pred, agg.by="calendar year")


preds_lcNa <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNa <- aggregateSolute(preds_lcNa,meta, format="flux total", se.preds=preds_lrNa$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemNa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcNaConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNaConc <- aggregateSolute(preds_lcNaConc,meta, format="conc", se.preds=preds_lcNaConc$se.pred, agg.by="calendar year")



datelims <- c("2011-01-01","2014-01-01")

date <- preds_lrNa[preds_lrNa$date >= as.POSIXct("2011-01-01") 
                                               & preds_lrNa$date <= as.POSIXct("2014-01-01"),]
MPR_chemNaShort <-MPR_chemNa[MPR_chemNa$date >= as.POSIXct("2011-01-01") 
                            & MPR_chemNa$date <= as.POSIXct("2014-01-01"),]
preds_lrNaShort <-preds_lrNaConc[preds_lrNaConc$date >= as.POSIXct("2011-01-01") 
                                 & preds_lrNaConc$date <= as.POSIXct("2014-01-01"),]
preds_lcNaShort <-preds_lcNaConc[preds_lcNaConc$date >= as.POSIXct("2011-01-01") 
                             & preds_lcNaConc$date <= as.POSIXct("2014-01-01"),]
MPRDischargeShort2<- MPRDischargeShort[MPRDischargeShort$date >= as.POSIXct("2011-01-01") 
                                           & MPRDischargeShort$date <= as.POSIXct("2014-01-01"),]
startDate <- as.POSIXct("2011-01-01") 
endDate <- as.POSIXct("2014-01-011")
fluxvar <- "Na_mg_L"
fluxName <- "Sodium"
#graphShortFlux <- function(preds_lr, preds_lc,  startDate, endDate, fluxName, 
#                           fluxvar, UNH_Sonadora_Field3){
graphShortFlux(preds_lrNa, preds_lcNa,  startDate, endDate, fluxName, 
                           fluxvar, MPRDischargeShort)


title <- "Sodium conc mg/L Rio Piedras"

#observed vs predicted rloadest 
ggplot() + ggtitle(title) + 
  geom_line(data= preds_lcNaConc, aes(x=date, y=fit , color="loadflex conc mg/L")) +
  #geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
  geom_point(data= MPR_chemNa, aes(x=date , y=Na_mg_L, color="observations conc mg/L"))+
  geom_line(data= MPRDischargeShort, aes(x=date, y=CFS,  color="cfs")) +
  geom_point(data= MPR_chemNa, aes(x=date, y=CFS,  color="cfs")) 

#observed vs predicted loadflex 
ggplot() + ggtitle(title) + 
  geom_line(data= preds_lrNaConc, aes(x=date, y=fit , color="loadflex conc mg/L")) +
  #geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
  geom_point(data= MPR_chemNa, aes(x=date , y=Na_mg_L, color="observations conc mg/L"))


ggplot() + ggtitle(title) + 
  geom_line(data= preds_lrNaShort, aes(x=date, y=fit , color="loadest conc mg/L")) +
  geom_line(data= preds_lcNaShort, aes(x=date, y=fit,  color="loadflex conc mg/L rectangular inteMPR")) +
  geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
  geom_point(data= MPR_chemNaShort, aes(x=date , y=Na_mg_L, color="observations conc mg/L"))

#Ca
MPR_chemCa <- MPR_chem[complete.cases(MPR_chem["Ca_mg_L"]),]
MPR_chemCa <- MPR_chemCa[complete.cases(MPR_chemCa["CFS"]),]

meta <- metadata(constituent="Ca_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(Ca_mg_L ~ model(1), data=MPR_chemCa,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemCa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrCa <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrCa <- aggregateSolute(preds_lrCa,meta, format="flux total", se.preds=preds_lrCa$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemCa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcCaConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcCaConc <- aggregateSolute(preds_lcCaConc,meta, format="conc", se.preds=preds_lcCaConc$se.pred, agg.by="calendar year")

#Mg
MPR_chemMg <- MPR_chem[complete.cases(MPR_chem["Mg_mg_L"]),]
MPR_chemMg <- MPR_chemMg[complete.cases(MPR_chemMg["CFS"]),]

meta <- metadata(constituent="Mg_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(Mg_mg_L ~ model(1), data=MPR_chemMg,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemMg, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrMg <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrMg <- aggregateSolute(preds_lrMg,meta, format="flux total", se.preds=preds_lrMg$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemMg, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcMgConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcMgConc <- aggregateSolute(preds_lcMgConc,meta, format="conc", se.preds=preds_lcMgConc$se.pred, agg.by="calendar year")


#NO3
MPR_chemNO3 <- MPR_chem[complete.cases(MPR_chem["NO3.N_ug_L"]),]
MPR_chemNO3 <- MPR_chemNO3[complete.cases(MPR_chemNO3["CFS"]),]

meta <- metadata(constituent="NO3.N_ug_L", flow="CFS", 
                 dates="date", conc.units="ug/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(NO3.N_ug_L ~ model(1), data=MPR_chemNO3,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemNO3, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcNO3Conc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNO3Conc <- aggregateSolute(preds_lcNO3Conc,meta, format="conc", se.preds=preds_lcNO3Conc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemNO3, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrNO3 <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrNO3 <- aggregateSolute(preds_lrNO3,meta, format="flux total", se.preds=preds_lrNO3$se.pred, agg.by="calendar year")
lrforreport <- loadReg(NO3.N_ug_L ~ model(1), data=MPR_chemNO3,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg")
loadReport(lrforreport,"NO3MPR")



#SO4
MPR_chemSO4 <- MPR_chem[complete.cases(MPR_chem["SO4.S_mg_L"]),]
MPR_chemSO4 <- MPR_chemSO4[complete.cases(MPR_chemSO4["CFS"]),]
range(MPR_chemSO4$SO4.S_mg_L)

ggplot(MPR_chemSO4, aes(x=SO4.S_mg_L)) +
  geom_histogram(binwidth=.2, colour="black", fill="white")

meta <- metadata(constituent="SO4.S_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(SO4.S_mg_L ~ model(1), data=MPR_chemSO4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))


lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemSO4, abs.or.rel.resids="absolute", interp.function=triangularInterpolation)

preds_lcSO4Conc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcSO4Conc <- aggregateSolute(preds_lcSO4Conc,meta, format="conc", se.preds=preds_lcSO4Conc$se.pred, agg.by="calendar year")

#try lr 

preds_lrSO4Conc <- predictSolute(lr, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrSO4Conc <- aggregateSolute(preds_lrSO4Conc,meta, format="conc", se.preds=preds_lcSO4Conc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemSO4, abs.or.rel.resids="relative", interp.function=triangularInterpolation)

preds_lrSO4 <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrSO4 <- aggregateSolute(preds_lrSO4,meta, format="flux total", se.preds=preds_lrSO4$se.pred, agg.by="calendar year")
lrforreport <- loadReg(SO4.S_mg_L ~ model(1), data=MPR_chemSO4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg")
loadReport(lrforreport,"SO4MPR")



title <- "SO4 conc mg/L MPR"
#observed vs predicted rloadest 
ggplot() + ggtitle(title) + 
  geom_line(data= preds_lcSO4Conc, aes(x=date, y=fit , color="loadflex conc mg/L")) +
  #geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
  geom_point(data= MPR_chemSO4, aes(x=date , y=SO4.S_mg_L, color="observations conc mg/L"))


#F
MPR_chemF <- MPR_chem[complete.cases(MPR_chem["Fluoride_mg_L"]),]
MPR_chemF <- MPR_chemF[complete.cases(MPR_chemF["CFS"]),]
MPRDischargeShort2<- MPRDischargeShort[MPRDischargeShort$date >= as.POSIXct("2013-01-01") 
                                                     & MPRDischargeShort$date <= as.POSIXct("2014-12-31"),]

meta <- metadata(constituent="Fluoride_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(Fluoride_mg_L ~ model(1), data=MPR_chemF,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))


lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemF, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lcFConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort2, se.pred=TRUE, date=TRUE)

aggs_lcFConc <- aggregateSolute(preds_lcFConc,meta, format="conc", se.preds=preds_lcFConc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemF, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lrF <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort2, se.pred=TRUE, date=TRUE)

#get some really high wierd values just remove them. 
#preds_lrF <-preds_lrF[preds_lrF$fit < 50,]

aggs_lrF <- aggregateSolute(preds_lrF,meta, format="flux total", se.preds=preds_lrF$se.pred, agg.by="calendar year")
lrforreport <- loadReg(Fluoride_mg_L ~ model(1), data=MPR_chemF,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg")
loadReport(lrforreport,"FMPR")

#try loadest

preds_lrFLoadestConc <- predictSolute(lr, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)
preds_lrFLoadest <- predictSolute(lr, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)
aggs_lrFLoadest <- aggregateSolute(preds_lrFLoadest,meta, format="flux total", se.preds=preds_lrFLoadest$se.pred, agg.by="calendar year")
aggs_lrFLoadestConc <- aggregateSolute(preds_lrFLoadestConc,meta, format="conc", se.preds=preds_lrFLoadestConc$se.pred, agg.by="calendar year")

#try triangular interpolation



lr <- loadReg2(loadReg(Fluoride_mg_L ~ model(1), data=MPR_chemF,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))


lcConctri <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemF, abs.or.rel.resids="relative", interp.function=triangularInterpolation)

preds_lcFConctri <- predictSolute(lcConctri, "conc",interval="prediction", MPRDischargeShort2, se.pred=TRUE, date=TRUE)

aggs_lcFConctri <- aggregateSolute(preds_lcFConctri,meta, format="conc", se.preds=preds_lcFConctri$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemF, abs.or.rel.resids="relative", interp.function=triangularInterpolation)

preds_lrFtri <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort2, se.pred=TRUE, date=TRUE)

aggs_lrFtri <- aggregateSolute(preds_lrFtri,meta, format="flux total", se.preds=preds_lrFtri$se.pred, agg.by="calendar year")

MPR_chemFShort <-MPR_chemF[MPR_chemF$date >= as.POSIXct("2013-01-01") 
                             & MPR_chemF$date <= as.POSIXct("2014-12-31"),]

preds_lcFShort <-preds_lcFConc[preds_lcFConc$date >= as.POSIXct("2013-01-01") 
                                   & preds_lcFConc$date <= as.POSIXct("2014-12-31"),]
preds_lcFShorttri <-preds_lcFConctri[preds_lcFConctri$date >= as.POSIXct("2013-01-01") 
                                         & preds_lcFConctri$date <= as.POSIXct("2014-12-31"),]
preds_lrFShortConc <-preds_lrFLoadestConc[preds_lrFLoadestConc$date >= as.POSIXct("2013-01-01") 
                               & preds_lrFLoadestConc$date <= as.POSIXct("2014-12-31"),]
preds_lrFShort<-preds_lrF[preds_lrF$date >= as.POSIXct("2013-01-01") 
                                     & preds_lrF$date <= as.POSIXct("2014-12-31"),]

# preds_lrFwrong <-preds_lrF[preds_lrF$fit >= 10000,]
# 
# preds_lrFRight <-preds_lrF[preds_lrF$fit < 50,]
# 
# preds_lrFShort<-preds_lrFRight[preds_lrFRight$date >= as.POSIXct("2013-01-01") 
#                           & preds_lrFRight$date <= as.POSIXct("2014-12-31"),]
# 
# 
# startDate <- as.POSIXct("2013-01-01") 
# endDate <- as.POSIXct("2014-01-011")
# fluxvar <- "Fluoride_mg_L"
# fluxName <- "F"
# 
# ggplot() +geom_point(data= MPRDischargeShort, aes(x=date, y=CFS,  color="cfs")) 
# title <- "F conc mg/L MPR loadflex"
# 
# #observed vs predicted rloadflex 
# ggplot() + ggtitle(title) + 
#   geom_line(data= preds_lcFShort, aes(x=date, y=fit , color="loadflex conc mg/L")) +
#   #geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
#   geom_point(data= MPR_chemF, aes(x=date , y=Fluoride_mg_L, color="observations conc mg/L"))
# 
# title <- "F Kg fluxes MPR loadflex"
# #observed vs predicted rloadflex 
# ggplot() + ggtitle(title) + 
#   #geom_line(data= preds_lcFShort, aes(x=date, y=fit , color="loadflex conc mg/L")) +
#   geom_line(data= preds_lrFShort, aes(x=date, y=fit , color="loadflex F flux kg")) +
#   #geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
#   geom_point(data= MPR_chemF, aes(x=date , y=F.Kg, color="observations Kg"))
# 
# 
# #observed vs predicted loadest 
# ggplot() + ggtitle(title) + 
#   geom_line(data= preds_lrFShort, aes(x=date, y=fit , color="loadest conc mg/L")) +
#   #geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
#   geom_point(data= MPR_chemF, aes(x=date , y=Fluoride_mg_L, color="observations conc mg/L"))
# 
# 
# ggplot() + ggtitle(title) + 
#   geom_line(data= preds_lrFShort, aes(x=date, y=fit , color="loadest conc mg/L")) +
#   geom_line(data= preds_lcFShort, aes(x=date, y=fit,  color="loadflex conc mg/L rectangular interp")) +
#   geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
#   geom_point(data= MPR_chemFShort, aes(x=date , y=Fluoride_mg_L, color="observations conc mg/L",size =6))
# 


#SiO2
MPR_chemSiO2 <- MPR_chem[complete.cases(MPR_chem["SiO2_mg_L"]),]
MPR_chemSiO2 <- MPR_chemSiO2[complete.cases(MPR_chemSiO2["CFS"]),]

meta <- metadata(constituent="SiO2_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(SiO2_mg_L ~ model(1), data=MPR_chemSiO2,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemSiO2, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcSiO2Conc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcSiO2Conc <- aggregateSolute(preds_lcSiO2Conc,meta, format="conc", se.preds=preds_lcSiO2Conc$se.pred, agg.by="calendar year")




lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemSiO2, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrSiO2 <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrSiO2 <- aggregateSolute(preds_lrSiO2,meta, format="flux total", se.preds=preds_lrSiO2$se.pred, agg.by="calendar year")




#PO4
MPR_chemPO4 <- MPR_chem[complete.cases(MPR_chem["PO4.P_ug_P_L"]),]
MPR_chemPO4 <- MPR_chemPO4[complete.cases(MPR_chemPO4["CFS"]),]

meta <- metadata(constituent="PO4.P_ug_P_L", flow="CFS", 
                 dates="date", conc.units="ug/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(PO4.P_ug_P_L ~ model(1), data=MPR_chemPO4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemPO4, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lcPO4Conc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcPO4Conc <- aggregateSolute(preds_lcPO4Conc,meta, format="conc", se.preds=preds_lcPO4Conc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemPO4, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lrPO4 <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrPO4 <- aggregateSolute(preds_lrPO4,meta, format="flux total", se.preds=preds_lrPO4$se.pred, agg.by="calendar year")

#boxplot(PO4.P_ug_P_L~log(CFS),data=MPR_chemPO4, main="", #
        xlab="log(cfs)", ylab="PO4")


lrforreport <- loadReg(PO4.P_ug_P_L ~ model(1), data=MPR_chemPO4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg")
loadReport(lrforreport,"PO4MPR")



#NH4
MPR_chemNH4 <- MPR_chem[complete.cases(MPR_chem["NH4.N_ug_L"]),]
MPR_chemNH4 <- MPR_chemNH4[complete.cases(MPR_chemNH4["CFS"]),]

meta <- metadata(constituent="NH4.N_ug_L", flow="CFS", 
                 dates="date", conc.units="ug/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(NH4.N_ug_L ~ model(1), data=MPR_chemNH4,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="ug/L", load.units="kg"))
lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemNH4, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lcNH4Conc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNH4Conc <- aggregateSolute(preds_lcNH4Conc,meta, format="conc", se.preds=preds_lcNH4Conc$se.pred, agg.by="calendar year")

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemNH4, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lrNH4 <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrNH4 <- aggregateSolute(preds_lrNH4,meta, format="flux total", se.preds=preds_lrNH4$se.pred, agg.by="calendar year")
# lrforreport <- loadReg(NH4.N_ug_L ~ model(1), data=MPR_chemNH4,
#                        flow="CFS", dates="date", time.step="instantaneous", 
#                        flow.units="cfs", conc.units="ug/L", load.units="kg")
# loadReport(lrforreport,"NH4MPR")

ggplot(MPR_chemPO4, aes(x=PO4.P_ug_P_L)) +
  geom_histogram(binwidth=.5, colour="black", fill="white")

ggplot(MPR_chemPO4, aes(x=NH4.N_ug_L)) +
  geom_histogram(binwidth=.5, colour="black", fill="white")


ggplot(MPR_chemPO4, aes(x=log(NH4.N_ug_L))) +
  geom_histogram(binwidth=.5, colour="black", fill="white")
shapiro.test(log(MPR_chemPO4$NH4.N_ug_L))
qqnorm(MPR_chemPO4$NH4.N_ug_L)

ggplot(MPR_chemPO4, aes(x=Mg_mg_L)) +
  geom_histogram(binwidth=.5, colour="black", fill="white")
qqnorm(MPR_chemPO4$Mg_mg_L)

ggplot() + ggtitle(title) + 
  geom_line(data= preds_lrNaShort, aes(x=date, y=fit , color="loadest conc mg/L")) +
  geom_line(data= preds_lcNaShort, aes(x=date, y=fit,  color="loadflex conc mg/L rectangular inteMPR")) +
  geom_line(data= MPRDischargeShort2, aes(x=date, y=log(CFS)*3,  color="cfs")) +
  geom_point(data= MPR_chemNaShort, aes(x=date , y=Na_mg_L, color="observations conc mg/L"))

#TDN
MPR_chemTDN <- MPR_chem[complete.cases(MPR_chem["TDN_mg_L"]),]
MPR_chemTDN <- MPR_chemTDN[complete.cases(MPR_chemTDN["CFS"]),]

meta <- metadata(constituent="TDN_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(TDN_mg_L ~ model(1), data=MPR_chemTDN,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemTDN, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrTDN <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrTDN <- aggregateSolute(preds_lrTDN,meta, format="flux total", se.preds=preds_lrTDN$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemTDN, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcTDNConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcTDNConc <- aggregateSolute(preds_lcTDNConc,meta, format="conc", se.preds=preds_lcTDNConc$se.pred, agg.by="calendar year")


#DON
MPR_chemDON <- MPR_chem[complete.cases(MPR_chem["DON_mg_L"]),]
MPR_chemDON <- MPR_chemDON[complete.cases(MPR_chemDON["CFS"]),]

meta <- metadata(constituent="DON_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(DON_mg_L ~ model(1), data=MPR_chemDON,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemDON, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lrDON <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrDON <- aggregateSolute(preds_lrDON,meta, format="flux total", se.preds=preds_lrDON$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemDON, abs.or.rel.resids="absolute", interp.function=rectangularInterpolation)

preds_lcDONConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcDONConc <- aggregateSolute(preds_lcDONConc,meta, format="conc", se.preds=preds_lcDONConc$se.pred, agg.by="calendar year")


ggplot(MPR_chemDON, aes(x=DON_mg_L)) +
  geom_histogram(binwidth=.02, colour="black", fill="white")

#DOC
MPR_chemDOC <- MPR_chem[complete.cases(MPR_chem["DOC_mg_L"]),]
MPR_chemDOC <- MPR_chemDOC[complete.cases(MPR_chemDOC["CFS"]),]

meta <- metadata(constituent="DOC_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(DOC_mg_L ~ model(1), data=MPR_chemDOC,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemDOC, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrDOC <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrDOC <- aggregateSolute(preds_lrDOC,meta, format="flux total", se.preds=preds_lrDOC$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemDOC, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcDOCConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcDOCConc <- aggregateSolute(preds_lcDOCConc,meta, format="conc", se.preds=preds_lcDOCConc$se.pred, agg.by="calendar year")


#Cl
MPR_chemCl <- MPR_chem[complete.cases(MPR_chem["Cl_mg_L"]),]
MPR_chemCl <- MPR_chemCl[complete.cases(MPR_chemCl["CFS"]),]

meta <- metadata(constituent="Cl_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(Cl_mg_L ~ model(1), data=MPR_chemCl,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemCl, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrCl <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrCl <- aggregateSolute(preds_lrCl,meta, format="flux total", se.preds=preds_lrCl$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemCl, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcClConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcClConc <- aggregateSolute(preds_lcClConc,meta, format="conc", se.preds=preds_lcClConc$se.pred, agg.by="calendar year")


#K
MPR_chemK <- MPR_chem[complete.cases(MPR_chem["K_mg_L"]),]
MPR_chemK <- MPR_chemK[complete.cases(MPR_chemK["CFS"]),]

meta <- metadata(constituent="K_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="MPR, PR")


lr <- loadReg2(loadReg(K_mg_L ~ model(1), data=MPR_chemK,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=MPR_chemK, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrK <- predictSolute(lc, "flux",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrK <- aggregateSolute(preds_lrK,meta, format="flux total", se.preds=preds_lrK$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=MPR_chemK, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcKConc <- predictSolute(lcConc, "conc",interval="prediction", MPRDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcKConc <- aggregateSolute(preds_lcKConc,meta, format="conc", se.preds=preds_lcKConc$se.pred, agg.by="calendar year")

